# SWB analysis pipeline part 4b: 

Freq-Band Averaged Time Courses - Single ROI

*Created: 09/30/2024* \
*Updated: 10/05/2024*


In [1]:
import numpy as np
import mne
from glob import glob
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
from scipy.stats import zscore, linregress, ttest_ind, ttest_rel, ttest_1samp, pearsonr, spearmanr
import pandas as pd
from mne.preprocessing.bads import _find_outliers
import os 
import joblib
import re
import datetime
import scipy
import random
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.regression.mixed_linear_model import MixedLM 
from joblib import Parallel, delayed
import pickle
import itertools
import time 
from matplotlib.ticker import StrMethodFormatter
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
import matplotlib as mpl
from matplotlib import cm



import warnings
warnings.filterwarnings('ignore')




In [7]:
# Specify root directory for un-archived data and results 
base_dir   = '/sc/arion/projects/guLab/Alie/SWB/'
anat_dir   = f'{base_dir}ephys_analysis/recon_labels/'
neural_dir = f'{base_dir}ephys_analysis/data/'
behav_dir  = f'{base_dir}swb_behav_models/data/behavior_preprocessed/'
script_dir = '/hpc/users/finka03/swb_ephys_analysis/scripts/'

date = datetime.date.today().strftime('%m%d%Y')
print(date)

10052024


In [8]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'{base_dir}ephys_analysis/LFPAnalysis/')

from LFPAnalysis import analysis_utils

sys.path.append(f'{script_dir}analysis_notebooks/')

from ieeg_tools import *

sys.path.append(f'{script_dir}behav/')

from behav_utils import *
from swb_subj_behav import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
subj_ids = list(pd.read_excel(f'{base_dir}ephys_analysis/subj_info/SWB_subjects.xlsx', sheet_name='Usable_Subjects', usecols=[0]).PatientID)
n_subj = len(subj_ids)
# subj_ids


In [10]:
bdi_list = pd.read_excel(f'{base_dir}ephys_analysis/subj_info/SWB_subjects.xlsx', sheet_name='Usable_Subjects', usecols=[3])
bdi_list = list(bdi_list.SWB_BDI)
subj_info_df = pd.DataFrame({'subj_id':subj_ids,'bdi':bdi_list})
subj_info_df



,subj_id,bdi
0,MS002,14
1,MS003,8
2,MS009,16
3,MS011,13
4,MS015,26
5,MS016,10
6,MS017,26
7,MS019,12
8,MS022,10
9,MS024,16


In [11]:
### load ROI reref master 
roi_reref_labels_master_df = pd.read_csv(glob(f'{base_dir}ephys_analysis/results/roi_info/roi_reref_labels_master.csv')[0])
roi_reref_labels_master_df = roi_reref_labels_master_df.drop(columns=['Unnamed: 0'])
roi_reref_labels_master_df



,subj_id,reref_ch_names,ch_label4roi,ch_type4roi,loc4roi,mni_x,mni_y,mni_z,hemi,roi
0,MS002,lacas1-lacas2,lacas1,anode,left cingulate gyrus d,-6.382462,37.158688,-3.130044,l,acc
1,MS002,lacas2-lacas3,lacas2,anode,left cingulate gyrus e,-6.368174,38.606223,2.270621,l,acc
2,MS002,lacas3-lacas4,lacas3,anode,left cingulate gyrus f,-6.390079,39.941566,7.640265,l,acc
3,MS002,lacas4-lacas5,lacas4,anode,left cingulate gyrus f,-6.914519,41.546899,12.993427,l,acc
4,MS002,lacas5-lacas6,lacas5,anode,left cingulate gyrus g,-6.933604,42.745784,18.267675,l,acc
...,...,...,...,...,...,...,...,...,...,...
1875,DA039,rtp6-rtp7,rtp6,anode,right superior middle temporal pole d,40.532268,7.354525,-33.253436,r,temporal pole
1876,DA039,rtp7-rtp8,rtp7,anode,right superior middle temporal pole d,43.911226,7.341369,-31.915085,r,temporal pole
1877,DA039,rsgcc6-rsgcc7,rsgcc7,cathode,right anterior pars triangularis b,42.378987,31.277597,2.471854,r,vlpfc
1878,DA039,rsgcc7-rsgcc8,rsgcc7,anode,right anterior pars triangularis b,42.378987,31.277597,2.471854,r,vlpfc


In [14]:
# all_behav = pd.read_csv(f'{behav_dir}all_behav.csv') ## this isn't normalized yet 
raw_behav = [pd.read_csv(f'{base_dir}ephys_analysis/behav/behav_data/{subj_id}_task_df.csv') for subj_id in subj_ids]
all_behav,drops_data = format_all_behav(raw_behav,return_drops=True)
del raw_behav
drops_data


{'MS002': {'bad_epochs': [18, 75], 'bad_epochs_t1': [17, 74, 149]},
 'MS003': {'bad_epochs': [36, 63, 75, 129],
  'bad_epochs_t1': [35, 62, 74, 128, 149]},
 'MS009': {'bad_epochs': [75], 'bad_epochs_t1': [74, 149]},
 'MS011': {'bad_epochs': [30, 37, 39, 66, 75, 135],
  'bad_epochs_t1': [29, 36, 38, 65, 74, 134, 149]},
 'MS015': {'bad_epochs': [32,
   36,
   37,
   38,
   40,
   51,
   57,
   61,
   68,
   70,
   72,
   75,
   90,
   121,
   133,
   140],
  'bad_epochs_t1': [31,
   35,
   36,
   37,
   39,
   50,
   56,
   60,
   67,
   69,
   71,
   74,
   89,
   120,
   132,
   139,
   149]},
 'MS016': {'bad_epochs': [12,
   17,
   26,
   31,
   32,
   43,
   45,
   50,
   75,
   77,
   106,
   130,
   136,
   144],
  'bad_epochs_t1': [11,
   16,
   25,
   30,
   31,
   42,
   44,
   49,
   74,
   76,
   105,
   129,
   135,
   143,
   149]},
 'MS017': {'bad_epochs': [75], 'bad_epochs_t1': [74, 149]},
 'MS019': {'bad_epochs': [75, 89], 'bad_epochs_t1': [74, 88, 149]},
 'MS022': {'bad_

# Aggregate ROI TFR Data, split by BDI (include all freqs)

In [15]:
data_save_dir = f'{base_dir}ephys_analysis/results/time_resolved_roi_data/'

In [16]:
roi_list = ['dlpfc','vlpfc','dmpfc','ofc','ains','pins','acc','amy','hpc']     


In [17]:
for roi in roi_list: 
    roi_df = roi_reref_labels_master_df[roi_reref_labels_master_df.roi == roi]
    roi_subjs  = roi_df.subj_id.unique().tolist()
    roi_elecs  = {f'{subj_id}':roi_df[roi_df.subj_id == subj_id].reref_ch_names.tolist() for subj_id in roi_subjs}
    del roi_df,roi_subjs
    
    roi_dict = {f'{subj_id}':[] for subj_id in list(roi_elecs.keys())}
    
    for subj_id in list(roi_elecs.keys()):
        power_epochs = mne.time_frequency.read_tfrs(f'{neural_dir}{subj_id}/{subj_id}_CpeOnset-tfr.h5').pick(roi_elecs[subj_id])
        roi_dict[subj_id] = np.delete(power_epochs._data,list(drops_data[subj_id]['bad_epochs']),axis=0)
        del power_epochs

    roi_path = f'{data_save_dir}{roi}/'
    os.makedirs(roi_path, exist_ok=True)
    roi_fname = f'{roi_path}{roi}_all_subj_dict.pkl'
    pickle.dump(roi_dict, open(roi_fname,"wb"))
    
    del roi_dict

        
        

Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS002/MS002_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS003/MS003_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS009/MS009_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS011/MS011_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS015/MS015_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS016/MS016_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS017/MS017_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS019/MS019_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS022/MS022_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS024/MS024_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/dat

Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS050/MS050_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/DA037/DA037_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/DA039/DA039_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS002/MS002_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS003/MS003_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS009/MS009_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS015/MS015_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS017/MS017_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS024/MS024_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS028/MS028_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/dat

Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS016/MS016_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS017/MS017_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS019/MS019_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS022/MS022_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS024/MS024_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS025/MS025_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS026/MS026_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS027/MS027_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS028/MS028_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS029/MS029_CpeOnset-tfr.h5 ...
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/dat

# LOAD ROI/FREQ DATA

In [12]:
# freqs = {'hfa':np.array([70,200]),'gamma':np.array([30,70]),'beta':np.array([13,30]),
#          'alpha':np.array([8,13]),'theta':np.array([4,8])}

In [ ]:
# ### plot_roi = roi str according to roi_reref_labels_master
# plot_roi   = 'acc'
# ### plot_band = str id of band to plot
# plot_band  = 'hfa'
# ### freq_range = np array of upper/lower freq bounds of plot_band - either manual or subset from freqs above
# # freq_range = np.array([9,12])
# freq_range = freqs[plot_band]

In [ ]:
# #### add bdi and cpe split to behav df
# regret_types = ['gamble_bad','safe_bad']

# all_behav['cpe_split'] = all_behav['res_type'].apply(lambda x: 0 if (x == regret_types[0])|(
#                                                         x == regret_types[1]) else 1)
# # all_behav['bdi_split'] = all_behav['bdi'].apply(lambda x: 0 if x < 20 else 1)


In [ ]:
# del roi_df

In [ ]:
# roi_band_pow = []

# for subj_id in roi_subjs:
    
#     # load cpe power epochs for single subject 
# #     power_epochs = mne.time_frequency.read_tfrs(f'{neural_dir}{subj_id}/{subj_id}_CpeOnset-tfr.h5').crop(0,2.0)
#     power_epochs = mne.time_frequency.read_tfrs(f'{neural_dir}{subj_id}/{subj_id}_CpeOnset-tfr.h5')
# #     roi_epochs   = power_epochs.copy().pick_channels(roi_elecs[subj_id])
#     roi_epochs   = power_epochs.copy().pick(roi_elecs[subj_id])

#     del power_epochs
    
#     roi_pow_df  = roi_epochs.to_data_frame()
    
#     del roi_epochs
    
#     roi_band_df = roi_pow_df[(roi_pow_df.freq >= freq_range[0])&(roi_pow_df.freq <= freq_range[1])].groupby(
#                   ['epoch','time']).agg('mean',numeric_only=True).reset_index()
    
#     del roi_pow_df
    
#     roi_band_df  = roi_band_df.melt(id_vars=['epoch','time','freq'],
#                                     value_vars=roi_elecs[subj_id],
#                                     var_name='reref_ch_names', value_name='band_pow',
#                                     ignore_index = False)
    
    
#     subj_behav = all_behav[all_behav.subj_id == subj_id]

#     subj_df = pd.merge(roi_band_df, subj_behav,on='epoch')
    
#     del roi_band_df, subj_behav
    
#     # add roi info to df 
#     subj_roi_labels = roi_reref_labels_master_df[roi_reref_labels_master_df['subj_id']==subj_id][
#                                 ['subj_id','reref_ch_names','roi','hemi']]
    
#     subj_df = pd.merge(subj_df, subj_roi_labels,left_on=['reref_ch_names','subj_id'],
#                                         right_on=['reref_ch_names','subj_id'])#.reset_index(drop=True)
    
#     subj_df['unique_reref_ch'] = subj_df[['subj_id', 'reref_ch_names']].agg('_'.join, axis=1)
    
#     # drop bad epochs
#     subj_drops = drops_data[subj_id]
#     subj_df = subj_df[~subj_df['epoch'].isin(subj_drops)]

    
# #     roi_band_pow[subj_id] = subj_df
#     roi_band_pow.append(subj_df)
#     del subj_df 
    
     

    

In [ ]:
# roi_band_pow = pd.concat(roi_band_pow).reset_index(drop=True)

In [ ]:
# roi_plot_df.to_csv(f'{base_dir}ephys_analysis/results/timeseries_plot_data/{plot_roi}_{plot_band}_roi_plot_df.csv')


## Plot Data

In [ ]:
# sns.set_context('talk')
# sns.set_style('ticks')

# fig_dir   = f'{base_dir}ephys_analysis/figs/band_timeseries/{plot_band}/'
# os.makedirs(fig_dir,exist_ok=True)

# sem_colors = ['indianred','steelblue']
# labels     = ['Negative CPE', 'Positive CPE']

# sns.set_context('talk')


In [ ]:
# # roi_ylim = [-0.1,0.15]
# # roi_ylim = [-0.015,0.025]

# for bdi in roi_band_pow.bdi_thresh.unique().tolist():
    
# #     if bdi_ix == 0:
# #         bdi = 'low'
# #     else:
# #         bdi = 'high'

#     bdi_data = roi_band_pow[roi_band_pow.bdi_thresh == bdi].reset_index(drop=True)
    
#     bdi_data['power'] = bdi_data.groupby(['unique_reref_ch','epoch']).band_pow.rolling(
#         window=100).mean().reset_index(drop=True)
    
    
#     power_avg = bdi_data.groupby(['time','cpe_split']).agg(power_mean=('power', 'mean'), 
#                                     power_sem=('power', lambda x: np.std(x) / np.sqrt(len(x)))).reset_index()
    
#     fig_name = f'{fig_dir}{bdi}BDI_{plot_roi}_{plot_band}.pdf'
    
#     fig,ax = plt.subplots(1,1,figsize=(6,4),dpi=400)
    
#     plt.axhline(0, color='black', linestyle='--',linewidth=2,alpha=0.7)
# #     plt.ylim(roi_ylim)
    
#     # Plot the time resolved power with SEM shading
#     for i, cpe_split in enumerate(bdi_data['cpe_split'].unique()):
        
#         subset = power_avg[power_avg['cpe_split'] == cpe_split]
        
#         plt.plot(subset['time'], subset['power_mean'], label=labels[i],color=sem_colors[i])
        
#         ax.legend(frameon=False,loc='upper center', bbox_to_anchor=(0.5, 0.5,0,0.7),ncol=2,
#                 framealpha=0,fontsize='x-small',shadow=None,markerscale=4,handlelength=1)

#         plt.fill_between(subset['time'], subset['power_mean'] - subset['power_sem'], 
#              subset['power_mean'] + subset['power_sem'], color=sem_colors[i], alpha=0.4)
#         del subset


#     fig.tight_layout()
#     fig.suptitle(f'{bdi} BDI {plot_roi} {plot_band}')
# #     ax.set_title(f'{bdi}BDI {plot_roi} {plot_band}',fontsize=20)
#     plt.xlabel(r'Time (s)')
# #     plt.ylabel(fr'${plot_band}_{{zpow}}$')
#     plt.ylabel('Beta Power')
    
#     sns.despine()

#     plt.savefig(fig_name,dpi=400,format='pdf', metadata=None,
#     bbox_inches='tight', pad_inches=0.1,
#     facecolor='auto', edgecolor='auto',
#     backend=None)
    
#     del bdi_data,power_avg



In [ ]:


# for bdi_thresh in roi_band_pow.bdi_split.unique().tolist():
    
#     if bdi_thresh == 0:
#         bdi = 'low'
#     else:
#         bdi = 'high'

#     bdi_data = roi_band_pow[roi_band_pow.bdi_split == bdi_thresh].reset_index(drop=True)
    
# #     bdi_data['power'] = bdi_data.groupby(['subj_id','cpe_split']
# #                                         ).band_pow.rolling(window=100).mean().reset_index(drop=True)
    
#     bdi_data['power'] = bdi_data.groupby(['cpe_split']
#                                         ).band_pow.rolling(window=500).mean().reset_index(drop=True)
    
    
#     power_avg = bdi_data.groupby(['time','cpe_split']).agg(power_mean=('power', 'mean'), 
#                                     power_sem=('power', lambda x: np.std(x) / np.sqrt(len(x)))).reset_index()
# #     power_avg
#     roi_ylim = [np.round(np.min(power_avg.power_mean)-0.01,3),
#                 np.round(np.max(power_avg.power_mean)+0.01,3)]
    
#     fig_name = f'{fig_dir}{bdi}BDI_{plot_roi}_{plot_band}.pdf'
    
#     fig,ax    = plt.subplots(1,1,figsize=(6,4),dpi=400)
    
#     plt.axhline(0, color='black', linestyle='--',linewidth=2,alpha=0.7)
#     plt.ylim(roi_ylim)
    
#     # Plot the time resolved power with SEM shading
#     for i, cpe_split in enumerate(bdi_data['cpe_split'].unique()):
        
#         subset = power_avg[power_avg['cpe_split'] == cpe_split]
        
# #         subset['power'] = subset.band_pow.rolling(window=50).mean()
# #         power_avg = subset.groupby(['time']).agg(power_mean=('band_pow', 'mean'), 
# #                                                 power_sem=('band_pow', lambda x: np.std(x)/np.sqrt(len(x)))
# #                                                 ).reset_index()
# #         subset = bdi_data[bdi_data['cpe_split'] == cpe_split].groupby('time').agg('mean',numeric_only=True).reset_index()
        
# # #         subset['power'] = subset.band_pow.rolling(window=50).mean()
        
# #         power_sem = np.std(subset.band_pow)/np.sqrt(len(subset))
        
#         plt.plot(subset['time'], subset['power_mean'], label=labels[i],color=sem_colors[i])
        
#         ax.legend(frameon=False,loc='upper center', bbox_to_anchor=(0.5, 0.5,0,0.7),ncol=2,
#                 framealpha=0,fontsize='x-small',shadow=None,markerscale=4,handlelength=1)

#         plt.fill_between(subset['time'], subset['power_mean'] - subset['power_sem'], 
#              subset['power_mean'] + subset['power_sem'], color=sem_colors[i], alpha=0.4)
#         del subset


#     fig.tight_layout()
#     fig.suptitle(f'{bdi}BDI {plot_roi} {plot_band}')
# #     ax.set_title(f'{bdi}BDI {plot_roi} {plot_band}',fontsize=20)
#     plt.xlabel(r'Time (s)')
# #     plt.ylabel(fr'${plot_band}_{{zpow}}$')
#     plt.ylabel('Beta Power')
# #     plt.ylim(roi_ylim)
    
#     sns.despine()

#     plt.savefig(fig_name,dpi=400,format='pdf', metadata=None,
#     bbox_inches='tight', pad_inches=0.1,
#     facecolor='auto', edgecolor='auto',
#     backend=None)
    
#     del bdi_data,power_avg

